In [1]:
pip install -q -U transformers peft datasets bitsandbytes trl accelerate unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.3/359.3 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Sanity check to ensure we can run this in google collab

In [2]:
from unsloth import FastLanguageModel
print("Unsloth OK")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth OK


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Full fine tuning script

In [4]:
from __future__ import annotations

import json
import os
import random
from pathlib import Path
from typing import Dict, List, Optional, Sequence, Tuple, Any

# ============================
# CONFIG
# ============================

# EXP_FILE = "/content/drive/MyDrive/output_procedural_dpo.jsonl"  # or "/mnt/data/output.jsonl"
# Base directory for your DPO JSONL files
DPO_BASE_DIR = "/content/drive/MyDrive"

# If you still want to be able to train on a single combined file, keep this
COMBINED_EXP_FILE = f"{DPO_BASE_DIR}/output_procedural_dpo.jsonl"

# list of per-approach DPO files to include
EXP_FILES = [
      f"{DPO_BASE_DIR}/merged_dpo.jsonl", # replaces individual file loads
#     # First-pass shards
#     f"{DPO_BASE_DIR}/output_procedural_dpo.client_memo.jsonl",
#     f"{DPO_BASE_DIR}/output_procedural_dpo.minimal_think.jsonl",
#     f"{DPO_BASE_DIR}/output_procedural_dpo.investor_impression.jsonl",
#     f"{DPO_BASE_DIR}/output_procedural_dpo.policy_first.jsonl",
#     f"{DPO_BASE_DIR}/output_procedural_dpo.disclosure_focus.jsonl",
#     f"{DPO_BASE_DIR}/output_procedural_dpo.checklist_short.jsonl",
#     f"{DPO_BASE_DIR}/output_procedural_dpo.pipeline_short.jsonl",

#     # Second-pass (compressed) shards
#     f"{DPO_BASE_DIR}/output_procedural_dpo.client_memo_compress.jsonl",
#     f"{DPO_BASE_DIR}/output_procedural_dpo.minimal_think_compress.jsonl",
#     f"{DPO_BASE_DIR}/output_procedural_dpo.investor_impression_compress.jsonl",
#     f"{DPO_BASE_DIR}/output_procedural_dpo.policy_first_compress.jsonl",
#     f"{DPO_BASE_DIR}/output_procedural_dpo.disclosure_focus_compress.jsonl",
#     f"{DPO_BASE_DIR}/output_procedural_dpo.checklist_short_compress.jsonl",
#     f"{DPO_BASE_DIR}/output_procedural_dpo.pipeline_short_compress.jsonl",
]

OUTPUT_DIR = "/content/dpo_qwen3_4b_thinking_dpo"

# where to write the eval split (all DPO examples not used for training)
DPO_EVAL_SPLIT_FILE = f"{DPO_BASE_DIR}/output_procedural_dpo_eval.jsonl"

# Truncation logging
TRUNCATION_LOG = "dpo_truncation_log.jsonl"

#---- Training size / steps control ----
MAX_STEPS = 260
TARGET_TRAIN_STEPS = 100
PER_DEVICE_BATCH_SIZE = 2
GRADIENT_ACCUM_STEPS = 4
SEED = 42              # or any integer you like
TRAIN_FRACTION = 0.95   # 90% of DPO records for training, 10% for eval
EFFECTIVE_BATCH_SIZE = PER_DEVICE_BATCH_SIZE * GRADIENT_ACCUM_STEPS  # 8
TARGET_TRAIN_SIZE = TARGET_TRAIN_STEPS * EFFECTIVE_BATCH_SIZE        # 1440 examples

# Eval preview (console)
EVAL_PREVIEW_COUNT = 1

# Final eval file (we will sample only 10 eval examples into this file)
FINAL_EVAL_FILE = "final_eval_results.txt"
FINAL_EVAL_SAMPLE_COUNT = 1

# Split seed
SPLIT_SEED = 13

# Model
MODEL_ID = "Qwen/Qwen3-4B-Thinking-2507"

# Length limits (must match DPOConfig)
MAX_LENGTH = 4096          # full sequence: prompt + answer
MAX_PROMPT_LENGTH = 3072   # prompt only

# Generation length for eval / demos
GEN_MAX_NEW_TOKENS = 2048  # enough room for full CoT + critique + rewrite

# ============================
# IMPORTS (UNSLOTH FIRST)
# ============================

try:
    from unsloth import FastLanguageModel
    HAVE_UNSLOTH = True
    print("✅ Unsloth import OK.")
except Exception as e:
    HAVE_UNSLOTH = False
    print("⚠️ Unsloth not usable, falling back to plain HF. Reason:", repr(e))

import torch
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import DPOTrainer, DPOConfig


✅ Unsloth import OK.


In [5]:
SYSTEM_PROMPT = """
You are in-house counsel and compliance officer for an SEC-registered investment adviser (RIA).

Your job:
- Spot material issues under the SEC Marketing Rule and firm advertising policies.
- Explain them briefly for an internal reviewer.
- Provide a clean, client-ready rewrite.

You must ALWAYS answer in EXACTLY this structure, with nothing before <think> and nothing after </fixed_copy>:

<think>...</think>
<critique>...</critique>
<fixed_copy>...</fixed_copy>

======================
PATTERN TO COPY (STYLE)
======================

<think>
Okay, the main issues here are unsubstantiated “premier” language and a single 12% performance figure with no time period or risk context. Under the Marketing Rule, that creates risks around implied superiority and an unbalanced presentation of results. I’ll reframe the reputation claim as clearly subjective, tie the 12% figure to a specific historical period, and add a brief reminder that results vary and loss is possible.
</think>

<critique>
• “One of the nation’s premier firms” reads like an objective ranking without support, raising unsubstantiated claim concerns.
• “Returned 12% net of fees” presents a point-in-time result with no time period or historical framing.
• “Continue to provide value to clients” could imply ongoing outperformance without balancing risks or variability.
</critique>

<fixed_copy>
I am pleased to introduce this month’s featured partner, [REDACTED:ORG], a firm I have worked with for more than 25 years. I believe [REDACTED:ORG] offers a thoughtful, research-driven approach to portfolio management.

Over the past 12 months, their flagship strategy delivered a 12% return net of fees. This result is historical and does not guarantee future performance. All investment strategies involve the risk of loss and results will vary.

Advisory services are provided by [REDACTED:ORG], L.L.C., a registered investment adviser where required by law. Registration does not imply a certain level of skill or training.
</fixed_copy>

======================
GUIDELINES (BRIEF)
======================

<think>
- 1 short paragraph, usually 2–4 sentences (about 40–80 words).
- It’s fine to sound like a quick internal monologue (e.g., “Okay, the main issues are…”), but stay concise and on-topic.
- Focus on: (a) what the copy is trying to do, (b) the main rule / risk themes, and (c) your plan for the rewrite.
- Do NOT mention “the user”, “the model”, “this task”, “system prompt”, or your own formatting.
- Avoid long step-by-step narration and do not re-quote large chunks of the original copy.
</think>

<critique>
- 3–5 bullet points, max ~120 words total.
- Each bullet: quote or closely paraphrase a phrase from the original copy AND name the concern (e.g., unsubstantiated claim, performance without period, unbalanced risk/benefit, testimonial/endorsement, weak disclosure).
- No process talk, no mention of prompts, tags, or being an AI.
</critique>

<fixed_copy>
- 1–3 short paragraphs, 80–200 words total.
- Preserve the legitimate business goal of the copy.
- Remove or soften non-compliant language.
- Make any performance or benefit statements clearly historical, time-bounded where relevant, and balanced with risks.
- Add or clarify disclosures (registration, no guarantee, risk of loss, limits of examples) where needed.
- Do NOT mention <think>, <critique>, “original text”, “edits”, or “compliance review”.
</fixed_copy>
""".strip()

In [6]:
# ============================
# I/O + TEXT HELPERS
# ============================

def load_jsonl(path: Path) -> List[Dict]:
    records = []
    with path.open("r", encoding="utf-8") as f:
        for i, line in enumerate(f, start=1):
            line = line.strip()
            if not line:
                continue
            try:
                records.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"[WARN] Skipping bad line {i}: {e}")
                print("        First 200 chars:", repr(line[:200]))
    return records

def write_jsonl(path: Path, rows: List[Dict]) -> None:
    path = Path(path)
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

def to_text(value: Any, field_name: str) -> str:
    # Already a string
    if isinstance(value, str):
        return value

    # Structured SEC response: {"think": "...", "critique": "...", "fixed_copy": "..."}
    if isinstance(value, dict):
        if all(k in value for k in ("think", "critique", "fixed_copy")):
            return (
                f"<think>{value['think']}</think>\n"
                f"<critique>{value['critique']}\n</critique>"
                f"<fixed_copy>{value['fixed_copy']}</fixed_copy>"
            )

        # Common patterns e.g. {"text": "..."} / {"completion": "..."} / etc.
        parts = []
        for key in ["text", "completion", "output", "response", "answer", "content"]:
            v = value.get(key)
            if isinstance(v, str):
                parts.append(v)
        if parts:
            return "\n".join(parts)

        # Fallback: join all string-like values in dict
        for k, v in value.items():
            if isinstance(v, str):
                parts.append(f"{k}: {v}")
        if parts:
            return "\n".join(parts)

    # Sometimes a list of strings or list of dicts
    if isinstance(value, list) and value:
        first = value[0]
        if isinstance(first, str):
            return first
        if isinstance(first, dict):
            return to_text(first, field_name + "[0]")

    raise TypeError(
        f"Field '{field_name}' is not a string-like text structure. "
        f"Got type={type(value)} example={repr(value)[:200]}"
    )


def build_prompt_text(broken_text: str, tokenizer: AutoTokenizer) -> str:
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": f"Input Copy:\n{broken_text}"},
    ]
    return tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )


# ============================
# MODEL LOADING
# ============================

def get_model_and_tokenizer() -> Tuple[torch.nn.Module, AutoTokenizer]:
    max_seq_length = MAX_LENGTH

    # Unsloth path
    if torch.cuda.is_available() and HAVE_UNSLOTH:
        major, _ = torch.cuda.get_device_capability(0)
        is_ampere = major >= 8

        print("🚀 CUDA + Unsloth detected.")
        if is_ampere:
            print("💪 Ampere-class GPU detected. Using bfloat16 + 4-bit QLoRA.")
        else:
            print("🎛 Non-Ampere CUDA GPU detected. Using default dtype + 4-bit QLoRA.")

        dtype = "bfloat16" if is_ampere else None

        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name     = MODEL_ID,
            max_seq_length = max_seq_length,
            dtype          = dtype,
            load_in_4bit   = True,
        )

        model = FastLanguageModel.get_peft_model(
            model,
            r              = 16,
            lora_alpha     = 32,
            lora_dropout   = 0.05,
            target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
            bias           = "none",
            use_gradient_checkpointing = True,
        )

        tokenizer.pad_token = tokenizer.eos_token
        FastLanguageModel.for_training(model)
        return model, tokenizer

    # HF fallback
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
    tokenizer.pad_token = tokenizer.eos_token

    if torch.cuda.is_available():
        print("⚡ CUDA detected (no usable Unsloth). Using HF model on GPU.")
        major, _ = torch.cuda.get_device_capability(0)
        is_ampere = major >= 8
        dtype = torch.bfloat16 if is_ampere else torch.float16

        model = AutoModelForCausalLM.from_pretrained(
            MODEL_ID,
            torch_dtype=dtype,
            device_map="auto",
        )
        model.gradient_checkpointing_enable()
        return model, tokenizer

    if torch.backends.mps.is_available():
        print("🍎 MPS detected. Using native float16 HF model.")
        device = "mps"
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_ID,
            torch_dtype=torch.float16,
            attn_implementation="sdpa",
        ).to(device)
        model.gradient_checkpointing_enable()
        return model, tokenizer

    print("⚠️ CPU detected. This will be slow.")
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.float32,
    )
    model.gradient_checkpointing_enable()
    return model, tokenizer


# ============================
# SPLIT / EVAL HELPERS
# ============================

def extract_source_id(record: Dict) -> Optional[str]:
    meta = record.get("meta") or record.get("source_meta")
    if isinstance(meta, dict):
        if meta.get("source_file") and meta.get("source_line") is not None:
            return f"{meta['source_file']}:{meta['source_line']}"
        if meta.get("response_id"):
            return f"response:{meta['response_id']}"
    broken = record.get("broken")
    if broken:
        return f"text:{hash(str(broken))}"
    return None


def split_train_eval(
    records: List[Dict],
    train_fraction: float,
    seed: int,
):
    """
    Simple random split of the *already-filtered* DPO file.

    - Assumes every record in `records` is already a good DPO example
      (gold, scores == 5, well-formed tags).
    - Returns (train_records, eval_records) with roughly `train_fraction`
      of examples in train, rest in eval.
    """
    if not records:
        raise RuntimeError("Training file produced zero samples (no records loaded at all).")

    rng = random.Random(seed)
    indices = list(range(len(records)))
    rng.shuffle(indices)

    if not (0.0 < train_fraction < 1.0):
        raise ValueError(f"train_fraction must be in (0, 1), got {train_fraction}")

    n_total = len(indices)
    n_train = max(1, int(n_total * train_fraction))
    n_train = min(n_train, n_total - 1) if n_total > 1 else 1  # ensure at least 1 eval if possible

    train_set = set(indices[:n_train])

    train_records = [records[i] for i in indices if i in train_set]
    eval_records = [records[i] for i in indices if i not in train_set]

    print(
        f"📊 Split {n_total} DPO records into "
        f"{len(train_records)} train and {len(eval_records)} eval."
    )
    return train_records, eval_records

def run_evaluation(
    model,
    tokenizer,
    eval_samples: List[Dict],
    save_path: Optional[Path] = None,
    max_items_to_print: int = 3,
) -> None:
    print("\n" + "=" * 30)
    print("🔎 EVALUATION: DPO CHECK")
    print("=" * 30)
    model.eval()

    f = None
    if save_path is not None:
        save_path = Path(save_path)
        save_path.parent.mkdir(parents=True, exist_ok=True)
        f = save_path.open("w", encoding="utf-8")
        print(f"📝 Writing full eval outputs to: {save_path}")

    for idx, sample in enumerate(eval_samples, 1):
        broken_text = to_text(sample["broken"], "broken(eval)")
        prompt = build_prompt_text(broken_text, tokenizer)
        inputs = tokenizer(prompt, return_tensors="pt")
        device = next(model.parameters()).device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=GEN_MAX_NEW_TOKENS,
                do_sample=False,
            )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        source_id = sample.get("source_id")

        block = (
            f"=== EVAL #{idx}"
            + (f" | source_id={source_id}" if source_id else "")
            + " ===\n"
            f"INPUT (first 300 chars): {broken_text[:300]}\n\n"
            f"MODEL OUTPUT:\n{response.strip()}\n\n"
            + "-" * 80 + "\n"
        )

        if f is not None:
            f.write(block)

        if idx <= max_items_to_print:
            print(block)

    if f is not None:
        f.close()
        print("✅ Full eval outputs written.")

    print("\n" + "=" * 30)


In [7]:
# ============================
# TRAINING PIPELINE
# ============================

def main_train():
    # Decide which experiment paths to use:
    # - If EXP_FILES is non-empty, use those (multi-file mode).
    # - Otherwise, fall back to single EXP_FILE.
    if EXP_FILES:
        exp_paths = [Path(p) for p in EXP_FILES]
    else:
        exp_paths = [Path(EXP_FILE)]

    # Sanity check: make sure all paths exist
    missing = [str(p) for p in exp_paths if not p.exists()]
    if missing:
        raise FileNotFoundError(
            "The following experiment files were not found:\n  " + "\n  ".join(missing)
        )

    if len(exp_paths) == 1:
        print(f"🚀 Starting DPO Training on {exp_paths[0]}")
    else:
        print("🚀 Starting DPO Training on multiple files:")
        for p in exp_paths:
            print(f"  - {p}")

    # Load model + tokenizer once
    model, tokenizer = get_model_and_tokenizer()

    # Load and merge all records
    raw_records: List[Dict] = []
    for p in exp_paths:
        records = load_jsonl(p)
        print(f"Loaded {len(records)} preference pairs from {p}")
        raw_records.extend(records)

    print(f"📦 Total merged preference pairs: {len(raw_records)}")

    # 🧹 Filter out ad copies whose broken text has: "Hi [REDACTED:NAME]"
    def has_bad_salutation(rec: Dict) -> bool:
        broken = rec.get("broken")
        if broken is None:
            return False
        text = to_text(broken, "broken(filter)")
        text = text.lstrip()
        # Exclude anything that *contains* this greeting
        return "Hi [REDACTED:NAME]" in text

    before = len(raw_records)
    raw_records = [r for r in raw_records if not has_bad_salutation(r)]
    after = len(raw_records)
    print(f"🧹 Filtered out {before - after} records with 'Hi [REDACTED:NAME]-' in the input copy.")

    # Simple random split of the merged DPO records
    train_records, eval_records = split_train_eval(
        raw_records,
        train_fraction=TRAIN_FRACTION,
        seed=SEED,
    )

    # Save ALL eval records to a JSONL so you can use them for offline evaluation
    if eval_records:
        eval_path = Path(DPO_EVAL_SPLIT_FILE)
        write_jsonl(eval_path, eval_records)
        print(f"💾 Saved {len(eval_records)} eval DPO records to {eval_path}")
    else:
        print("⚠️ No eval records created – TRAIN_FRACTION is too high for dataset size.")

    eval_samples = [
        {"broken": rec["broken"], "source_id": extract_source_id(rec)}
        for rec in eval_records
    ]

    print(
        f"Selected {len(train_records)} examples for TRAIN "
        f"and {len(eval_samples)} examples for EVAL."
    )

    if not train_records:
        raise RuntimeError("No training data available after split.")

    formatted_data: List[Dict] = []
    truncation_meta: List[Dict] = []

    # Length-aware formatting
    for idx, r in enumerate(train_records):
        broken_text   = to_text(r["broken"],   "broken(train)")
        chosen_text   = to_text(r["chosen"],   "chosen(train)")
        rejected_text = to_text(r["rejected"], "rejected(train)")

        prompt = build_prompt_text(broken_text, tokenizer)

        prompt_ids   = tokenizer(prompt, add_special_tokens=False).input_ids
        chosen_ids   = tokenizer(chosen_text, add_special_tokens=False).input_ids
        rejected_ids = tokenizer(rejected_text, add_special_tokens=False).input_ids

        pl = len(prompt_ids)
        cl = len(chosen_ids)
        rl = len(rejected_ids)

        total_c = pl + cl
        total_r = pl + rl

        trunc_prompt  = pl > MAX_PROMPT_LENGTH
        trunc_chosen  = total_c > MAX_LENGTH
        trunc_reject  = total_r > MAX_LENGTH

        if trunc_prompt or trunc_chosen or trunc_reject:
            truncation_meta.append(
                {
                    "index": idx,
                    "source_id": extract_source_id(r),
                    "prompt_len": pl,
                    "chosen_len": cl,
                    "rejected_len": rl,
                    "total_prompt_chosen": total_c,
                    "total_prompt_rejected": total_r,
                    "trunc_prompt": trunc_prompt,
                    "trunc_chosen": trunc_chosen,
                    "trunc_rejected": trunc_reject,
                }
            )
            # Skip pair entirely to avoid hidden truncation
            continue

        formatted_data.append(
            {
                "prompt":  prompt,
                "chosen":  chosen_text,
                "rejected": rejected_text,
            }
        )

    # Truncation logging
    if truncation_meta:
        print(
            f"⚠️ Skipped {len(truncation_meta)} training pairs "
            f"because they exceeded MAX_PROMPT_LENGTH={MAX_PROMPT_LENGTH} "
            f"or MAX_LENGTH={MAX_LENGTH}."
        )
        with open(TRUNCATION_LOG, "w", encoding="utf-8") as f:
            for rec in truncation_meta:
                f.write(json.dumps(rec) + "\n")
        print(f"   Details written to: {TRUNCATION_LOG}")
        print("   First 3 skipped examples (lengths only):")
        for rec in truncation_meta[:3]:
            print(
                f"    idx={rec['index']}, src={rec['source_id']}, "
                f"prompt={rec['prompt_len']}, chosen={rec['chosen_len']}, "
                f"rejected={rec['rejected_len']}"
            )
    else:
        print("✅ No training pairs exceeded the length limits; no truncation expected.")

    print(f"Training on {len(formatted_data)} preference pairs AFTER length filtering.")

    if not formatted_data:
        raise RuntimeError("All training data was filtered out due to length limits.")

    train_dataset = Dataset.from_list(formatted_data)

    is_cuda = torch.cuda.is_available()
    is_ampere = False
    if is_cuda:
        major, _ = torch.cuda.get_device_capability(0)
        is_ampere = major >= 8

    # Rough info about steps if we *did* run a full epoch
    steps_per_epoch = (len(formatted_data) + EFFECTIVE_BATCH_SIZE - 1) // EFFECTIVE_BATCH_SIZE
    print(
        f"With effective batch size {EFFECTIVE_BATCH_SIZE}, "
        f"1 epoch would be ~{steps_per_epoch} steps."
    )
    print(f"Using max_steps={MAX_STEPS} – training will stop after ~{MAX_STEPS} optimizer updates.")

    training_args = DPOConfig(
        output_dir=OUTPUT_DIR,
        num_train_epochs=1,                # ignored when max_steps > 0,
        max_steps=MAX_STEPS,               # hard stop at this many optimizer steps
        per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUM_STEPS,
        learning_rate=5e-6,
        beta=0.2,
        logging_steps=10,
        save_strategy="epoch",
        save_total_limit=1,
        fp16=False if is_ampere else (is_cuda or torch.backends.mps.is_available()),
        bf16=is_ampere,
        optim="adamw_torch_fused" if is_ampere else "paged_adamw_8bit",
        gradient_checkpointing=True,
        max_length=MAX_LENGTH,
        max_prompt_length=MAX_PROMPT_LENGTH,
        report_to="none",
        remove_unused_columns=False,
        warmup_ratio=0.03,
        seed=SEED,
    )

    trainer = DPOTrainer(
        model=model,
        ref_model=None,
        train_dataset=train_dataset,
        processing_class=tokenizer,
        args=training_args,
    )

    print("💪 Training started...")
    trainer.train()
    trainer.save_model(OUTPUT_DIR)
    print(f"✅ Model saved to: {OUTPUT_DIR}")

    # --- Small preview eval to console ---
    if EVAL_PREVIEW_COUNT > 0 and eval_samples:
        preview_count = min(EVAL_PREVIEW_COUNT, len(eval_samples))
        preview_samples = random.sample(eval_samples, preview_count)
        print(f"✅ Eval preview on {len(preview_samples)} samples.")
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        run_evaluation(
            model,
            tokenizer,
            preview_samples,
            save_path=None,
            max_items_to_print=EVAL_PREVIEW_COUNT,
        )

    # --- Final eval: sample only 10 and write to file ---
    if eval_samples:
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

        final_eval_samples = random.sample(
            eval_samples,
            min(FINAL_EVAL_SAMPLE_COUNT, len(eval_samples)),
        )
        print(f"📄 Running final eval on {len(final_eval_samples)} sampled examples.")

        run_evaluation(
            model,
            tokenizer,
            final_eval_samples,
            save_path=Path(FINAL_EVAL_FILE),
            max_items_to_print=0,  # file-only
        )
        print(f"📄 Final eval results written to: {FINAL_EVAL_FILE}")

    print("🎉 DPO training complete.")


if __name__ == "__main__":
    main_train()

🚀 Starting DPO Training on /content/drive/MyDrive/merged_dpo.jsonl
🚀 CUDA + Unsloth detected.
💪 Ampere-class GPU detected. Using bfloat16 + 4-bit QLoRA.
==((====))==  Unsloth 2025.11.6: Fast Qwen3 patching. Transformers: 4.57.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.51G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

💪 Training started...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 943 | Num Epochs = 3 | Total steps = 260
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 11,796,480 of 4,034,264,576 (0.29% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss
10,0.692700,-0.006308,-0.015178,0.400000,0.008871,-695.593079,-678.649841,-4.176251,-4.200210,0,0,0
20,0.696800,0.124445,0.124896,0.487500,-0.000451,-688.832336,-643.399109,-4.203773,-4.237559,No Log,No Log,No Log
30,0.600900,0.400802,0.185705,0.750000,0.215096,-708.744263,-692.808594,-4.157671,-4.196826,No Log,No Log,No Log
40,0.534600,0.697483,0.285654,0.750000,0.411830,-706.951843,-761.040710,-4.203709,-4.224004,No Log,No Log,No Log
50,0.496000,0.991978,0.466524,0.762500,0.525454,-695.887634,-696.648315,-4.183319,-4.220083,No Log,No Log,No Log
60,0.418900,1.175464,0.370328,0.812500,0.805135,-687.898804,-658.435425,-4.155662,-4.185488,No Log,No Log,No Log
70,0.352300,1.625402,0.527837,0.850000,1.097564,-698.964722,-653.064636,-4.157451,-4.175231,No Log,No Log,No Log
80,0.300400,2.139779,0.611555,0.912500,1.528224,-695.787842,-701.567505,-4.169220,-4.196116,No Log,No Log,No Log
90,0.255400,1.984412,0.100132,0.887500,1.884280,-670.010193,-671.669739,-4.190693,-4.213547,No Log,No Log,No Log
100,0.199900,2.398305,0.180998,0.925000,2.217307,-692.960327,-657.487915,-4.139366,-4.185216,No Log,No Log,No Log


✅ Model saved to: /content/dpo_qwen3_4b_thinking_dpo
✅ Eval preview on 1 samples.

🔎 EVALUATION: DPO CHECK
=== EVAL #1 | source_id=output/synthetic/outputs-batch-2b24cd86-7d60-486f-a010-82281bb38c0c-prediction-model-2025-11-29T01-25-35.222530Z-predictions.jsonl:3151 ===
INPUT (first 300 chars): Based on the compliance feedback provided, here is the revised content for the social media post. The language has been adjusted to remove guarantees, the visual direction has been altered to avoid promissory implications, and the mandatory registration disclosure has been appended.

### Revised Soc

MODEL OUTPUT:
system
You are in-house counsel and compliance officer for an SEC-registered investment adviser (RIA).

Your job:
- Spot material issues under the SEC Marketing Rule and firm advertising policies.
- Explain them briefly for an internal reviewer.
- Provide a clean, client-ready rewrite.

You must ALWAYS answer in EXACTLY this structure, with nothing before <think> and nothing after </fi

In [8]:
def load_base_model_and_tokenizer() -> Tuple[Any, Any]:
    """
    Load the *base* Qwen3-4B-Thinking model (no LoRA) for comparison runs.
    Uses Unsloth if available, otherwise plain HF.
    """
    if HAVE_UNSLOTH:
        print("🔁 Loading BASE model via Unsloth (no LoRA).")
        model, tokenizer = FastLanguageModel.from_pretrained(
            MODEL_ID,
            dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
            load_in_4bit=False,   # set True if you want 4-bit for eval as well
        )
        FastLanguageModel.for_inference(model)
    else:
        print("🔁 Loading BASE model via plain HF (no LoRA).")
        tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_ID,
            torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
            device_map="auto",
        )

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer


def load_finetuned_model_and_tokenizer() -> Tuple[Any, Any]:
    """
    Load the fine-tuned DPO model from OUTPUT_DIR.
    Assumes trainer.save_model(OUTPUT_DIR) was used.
    """
    if HAVE_UNSLOTH:
        print("🔁 Loading FINETUNED model via Unsloth from:", OUTPUT_DIR)
        model, tokenizer = FastLanguageModel.from_pretrained(
            OUTPUT_DIR,
            dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
            load_in_4bit=False,
        )
        FastLanguageModel.for_inference(model)
    else:
        print("🔁 Loading FINETUNED model via plain HF from:", OUTPUT_DIR)
        tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR)
        model = AutoModelForCausalLM.from_pretrained(
            OUTPUT_DIR,
            torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
            device_map="auto",
        )

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer


# ---------------------------------------------------------
# Shared helper: run base + finetuned on a single text and
# save both outputs to disk.
# ---------------------------------------------------------
def run_base_and_ft_for_text(
    text: str,
    source_id: str,
    out_dir: Path,
    base_model,
    base_tokenizer,
    ft_model,
    ft_tokenizer,
    filename_prefix: str,
    sample_idx: Optional[int] = None,   # e.g., 0, 1, 2... for eval samples
    eval_index: Optional[int] = None,   # original index in eval set (if any)
) -> Tuple[Path, Path]:
    """
    Core utility that:
      - builds prompts for base + finetuned
      - runs generation for both
      - writes two files (base + finetuned) into `out_dir`
      - returns (base_file_path, ft_file_path)

    filename_prefix:
      - "eval" for eval records (e.g., eval_00_<source>_base.txt)
      - "custom" for ad-hoc inputs (e.g., custom_<source>_base.txt)
    """
    # Devices
    base_device = next(base_model.parameters()).device
    ft_device = next(ft_model.parameters()).device

    # Build prompts same way as in DPO
    prompt_base = build_prompt_text(text, base_tokenizer)
    prompt_ft   = build_prompt_text(text, ft_tokenizer)

    # For human inspection, they’ll be nearly identical; pick one to display
    display_prompt = prompt_ft

    # --- BASE MODEL GENERATION ---
    base_inputs = base_tokenizer(
        prompt_base,
        return_tensors="pt",
        add_special_tokens=False,
    )
    base_input_ids = base_inputs["input_ids"].to(base_device)

    with torch.no_grad():
        base_gen_ids = base_model.generate(
            input_ids=base_input_ids,
            max_new_tokens=GEN_MAX_NEW_TOKENS,
            do_sample=False,
            temperature=1.0,
            top_p=1.0,
            eos_token_id=base_tokenizer.eos_token_id,
        )[0]

    base_gen_only_ids = base_gen_ids[base_input_ids.shape[1]:]
    base_gen_text = base_tokenizer.decode(
        base_gen_only_ids,
        skip_special_tokens=True,
    )

    # --- FINETUNED MODEL GENERATION ---
    ft_inputs = ft_tokenizer(
        prompt_ft,
        return_tensors="pt",
        add_special_tokens=False,
    )
    ft_input_ids = ft_inputs["input_ids"].to(ft_device)

    with torch.no_grad():
        ft_gen_ids = ft_model.generate(
            input_ids=ft_input_ids,
            max_new_tokens=GEN_MAX_NEW_TOKENS,
            do_sample=False,
            temperature=1.0,
            top_p=1.0,
            eos_token_id=ft_tokenizer.eos_token_id,
        )[0]

    ft_gen_only_ids = ft_gen_ids[ft_input_ids.shape[1]:]
    ft_gen_text = ft_tokenizer.decode(
        ft_gen_only_ids,
        skip_special_tokens=True,
    )

    # --- File naming ---
    safe_source = str(source_id).replace("/", "_").replace(" ", "_")[:60]

    if sample_idx is not None:
        base_fname = out_dir / f"{filename_prefix}_{sample_idx:02d}_{safe_source}_base.txt"
        ft_fname   = out_dir / f"{filename_prefix}_{sample_idx:02d}_{safe_source}_ft.txt"
    else:
        base_fname = out_dir / f"{filename_prefix}_{safe_source}_base.txt"
        ft_fname   = out_dir / f"{filename_prefix}_{safe_source}_ft.txt"

    # --- Write BASE file ---
    with base_fname.open("w", encoding="utf-8") as f:
        f.write(f"source_id: {source_id}\n")
        if eval_index is not None:
            f.write(f"eval_index: {eval_index}\n")
        f.write("model: base\n\n")
        f.write("=== PROMPT ===\n")
        f.write(display_prompt)
        f.write("\n\n=== BASE MODEL RAW OUTPUT ===\n")
        f.write(base_gen_text)
        f.write("\n")

    # --- Write FINETUNED file ---
    with ft_fname.open("w", encoding="utf-8") as f:
        f.write(f"source_id: {source_id}\n")
        if eval_index is not None:
            f.write(f"eval_index: {eval_index}\n")
        f.write("model: finetuned\n\n")
        f.write("=== PROMPT ===\n")
        f.write(display_prompt)
        f.write("\n\n=== FINETUNED MODEL RAW OUTPUT ===\n")
        f.write(ft_gen_text)
        f.write("\n")

    return base_fname, ft_fname


# ---------------------------------------------------------
# Refactored: eval-sampling function now uses the helper
# ---------------------------------------------------------
def run_base_and_ft_on_eval_samples(
    sample_count: int = 10,
    seed: int = SPLIT_SEED,
) -> None:
    """
    Sample `sample_count` eval records from DPO_EVAL_SPLIT_FILE,
    run BOTH the base model and the fine-tuned model on each,
    and save each (prompt + raw output) as its own .txt file
    under `outputs_ara` next to the eval file.

    For each sampled eval record we create TWO files:
      - eval_XX_<source>_base.txt
      - eval_XX_<source>_ft.txt
    """
    eval_path = Path(DPO_EVAL_SPLIT_FILE)
    if not eval_path.exists():
        print(f"⚠️ Eval split file not found at {eval_path}; skipping eval sampling.")
        return

    with eval_path.open("r", encoding="utf-8") as f:
        eval_records = [json.loads(line) for line in f if line.strip()]

    if not eval_records:
        print("⚠️ Eval split is empty; nothing to run against models.")
        return

    rng = random.Random(seed)
    indices = list(range(len(eval_records)))
    rng.shuffle(indices)
    picked_indices = indices[: min(sample_count, len(indices))]

    print(f"🎯 Sampling {len(picked_indices)} eval records for base vs fine-tuned comparison.")

    # Load both models once
    base_model, base_tokenizer = load_base_model_and_tokenizer()
    ft_model, ft_tokenizer = load_finetuned_model_and_tokenizer()

    # Output dir: sibling of DPO_EVAL_SPLIT_FILE
    out_dir = eval_path.parent / "outputs_ara"
    out_dir.mkdir(parents=True, exist_ok=True)
    print(f"📝 Saving comparison outputs to: {out_dir}")

    for i, idx in enumerate(picked_indices):
        rec = eval_records[idx]
        broken_text = rec.get("broken", "")
        source_id = extract_source_id(rec)

        base_fname, ft_fname = run_base_and_ft_for_text(
            text=broken_text,
            source_id=source_id,
            out_dir=out_dir,
            base_model=base_model,
            base_tokenizer=base_tokenizer,
            ft_model=ft_model,
            ft_tokenizer=ft_tokenizer,
            filename_prefix="eval",
            sample_idx=i,      # for file name: eval_XX_...
            eval_index=idx,    # stored inside file as eval_index
        )

        print(f"   ✔ Wrote base file:      {base_fname}")
        print(f"   ✔ Wrote finetuned file: {ft_fname}")


# ---------------------------------------------------------
# Refactored: custom free-form input function using helper
# ---------------------------------------------------------
def run_base_and_ft_on_custom_copy(
    input_copy: str,
    source_id: str = "custom_input",
) -> None:
    """
    Run BOTH the base model and the fine-tuned model on an arbitrary input string
    and save each (prompt + raw output) as its own .txt file in the SAME
    outputs_ara directory used by `run_base_and_ft_on_eval_samples`.
    """
    # Load both models once
    base_model, base_tokenizer = load_base_model_and_tokenizer()
    ft_model, ft_tokenizer = load_finetuned_model_and_tokenizer()

    # Reuse the same output directory convention
    eval_path = Path(DPO_EVAL_SPLIT_FILE)
    out_dir = eval_path.parent / "outputs_ara"
    out_dir.mkdir(parents=True, exist_ok=True)
    print(f"📝 Saving custom comparison outputs to: {out_dir}")

    base_fname, ft_fname = run_base_and_ft_for_text(
        text=input_copy,
        source_id=source_id,
        out_dir=out_dir,
        base_model=base_model,
        base_tokenizer=base_tokenizer,
        ft_model=ft_model,
        ft_tokenizer=ft_tokenizer,
        filename_prefix="custom",
        sample_idx=None,
        eval_index=None,
    )

    print(f"   ✔ Wrote base file:      {base_fname}")
    print(f"   ✔ Wrote finetuned file: {ft_fname}")



In [9]:
import shutil

output_dir = "/content/dpo_qwen3_4b_thinking_dpo"
zip_file_path = Path(DPO_EVAL_SPLIT_FILE).parent / "dpo_fintuned"
shutil.make_archive(zip_file_path, 'zip', output_dir)
print(f"Zipped output directory '{output_dir}' to '{zip_file_path}.zip'")

Zipped output directory '/content/dpo_qwen3_4b_thinking_dpo' to '/content/drive/MyDrive/dpo_fintuned.zip'


Get a sample of 1 ad copies for human evals relative to baseline model

In [10]:
# After training is done and DPO_EVAL_SPLIT_FILE exists:
# run_base_and_ft_on_eval_samples(sample_count=1)

In [ ]:
# Assuming you have already defined:
# - run_base_and_ft_on_custom_copy(input_copy: str, source_id: str = "custom_input")
# - and all needed imports / constants from your existing script

synthetic_copies = [
    '''Platform: Facebook Ad

Firm: [REDACTED:ORG]

Headline: Retire 5 Years Sooner (Without Saving More)

Primary Text:
If you’re 50+ with at least $500,000 saved, our 5-Year FastTrack™ plan shows you the exact date you can retire – and we’ve never seen it be wrong.

Clients who follow our steps:
• Always retire at least 5 years earlier than they expected
• Never run out of money in retirement
• Consistently earn more than a basic index fund

Skip the jargon. Tap the button, book a 15-minute call, and we’ll tell you the year you can walk away… before we even see your full financials.

👉 Learn More: [REDACTED:URL]

Image suggestion: Smiling couple on a beach with overlaid text: “Retire 5 Years Sooner – Guaranteed With Our FastTrack™ Plan”''',

    '''Platform: Facebook Ad (Lead Form)

Firm: [REDACTED:ORG]

Headline: Your Heirs Are Owed More Than This

Primary Text:
Most families in [REDACTED:ADDRESS] are on track to give 30–40% of their life savings to taxes and legal fees when they pass away.

Our Estate Reset™ checklist fixes that in one meeting. Families who complete it:
• Avoid probate 100% of the time
• Eliminate estate taxes, regardless of portfolio size
• Keep every dollar with the people they choose – we’ve never seen a plan fail

You don’t need an attorney, CPA, and advisor. Just us.

Tap “Get Checklist” and we’ll send you the same document we use for our private clients.

✅ Get Checklist: [REDACTED:URL]

Disclaimer line (tiny in graphic): Results shown are typical for our implementation process.''',

    '''Platform: LinkedIn Post

Firm: [REDACTED:ORG]

Copy:
I just ran the numbers for a client who thought their HSA was “just a cash bucket for doctor visits.”

If they had followed our HSA Ramp™ approach from day one, here’s what they would have today:

• 6-figure HSA balance
• Tax-free growth that never had a down year
• A fully funded healthcare bucket that beats most 401(k) portfolios

Our internal research shows that 100% of clients who implement HSA Ramp™ and stick with it for 10 years end up with more retirement wealth than if they had simply maxed their 401(k).

No complicated decisions. We literally tell you which funds to pick in your HSA and when to rebalance – the same way we do it in-house.

Comment “HSA” and we’ll send the exact allocation we give paying clients.

#HSA #RetirementPlanning #[REDACTED:ORG]''',

    '''Platform: LinkedIn Short Post

Firm: [REDACTED:ORG]

Copy:
You don’t need another article about “why Roth IRAs are great.” You need someone to tell you yes or no.

At [REDACTED:ORG], our Roth Yes/No Matrix has produced the same clear answer for every client we’ve run it for – and we’ve yet to see a situation where it steers someone wrong.

If the Matrix says “Yes, convert,” you will end up wealthier in retirement. If it says “No,” you won’t.

It really is that simple when you stop overcomplicating the math.

DM us “MATRIX” and we’ll run your Roth Yes/No for free.

#RothIRA #TaxPlanning #[REDACTED:ORG]''',

    '''Platform: Facebook Post (Organic Page)

Firm: [REDACTED:ORG]

Copy:
🎯 Quick Question For 40–60 Year Olds 🎯

If you knew, with certainty, that changing ONE beneficiary form could add 6 figures to your children’s inheritance… would you do it?

Every family that completes our Beneficiary Clean-Up Call sees more money going to their heirs and less to taxes, period. We’ve never had a case where our recommendations didn’t improve the outcome.

No complex legal talk. No “it depends.” Just:
1️⃣ We review your statements
2️⃣ We tell you exactly what to change
3️⃣ You sign, and your legacy is fixed for good

Comment “HEIRS” to grab a spot on this week’s call.

#EstatePlanning #Legacy #[REDACTED:ORG]

Image suggestion: Before/after graphic labeled “Old Forms vs. Clean Forms” showing only the “after” tax result.''',

    '''Platform: Website – Home Page Hero + Sections

Site: [REDACTED:URL]
Firm: [REDACTED:ORG]

Hero Section:
Headline: The Only Retirement & Estate Plan You’ll Ever Need
Subheadline: We design portfolios and legal structures that beat the market, erase future taxes, and protect your family – with a process that has never failed a qualified client.
Primary CTA Button: Get My Forever Plan
Secondary CTA: See Our 3-Year Performance

Section 1 – How We’re Different

At [REDACTED:ORG], we don’t “hope” your plan works. We engineer it.

Our AllWeather Legacy™ framework combines:
• A portfolio that has outperformed its benchmark across every 3-year period we track
• A tax strategy that always produces lower lifetime taxes than a traditional CPA approach
• Attorney-grade estate documents that never need to be updated, regardless of future law changes

Every client who completes our process walks away with:
• A specific retirement date that we’ve never seen missed
• A guaranteed minimum income level for life
• Written proof that their heirs will avoid probate and estate taxes

Section 2 – Who We Serve

We work exclusively with households:
• $1M–$10M in investable assets
• Ready to delegate every financial, tax, and estate decision
• Who want outcomes, not options

If that’s you, our process works the same way every time – no guesswork, no uncertainty.

Section 3 – Proof It Works

We don’t ask you to trust us. We show you:
• Composite performance showing double-digit returns in every calendar year since launch
• Case studies where clients retired early and never experienced a negative year on their statements
• Real examples of zero-tax estates created using our documents

Regulators across multiple states have approved our registrations, effectively validating the strength of our process.

Footer Contact:
[REDACTED:ADDRESS] | [REDACTED:PHONE] | [REDACTED:EMAIL]''',

    '''Platform: Synthetic PDF – Guide Excerpt

Title: The 55/250 Playbook – How To Retire At 55 With $250,000 Of Annual Income (No Matter What The Market Does)
Author: [REDACTED:ORG]

Excerpt:

Introduction

Most retirement books talk about “rules of thumb” and “4% withdrawal guidelines.” We don’t.

The 55/250 Playbook is the exact system we use with private clients to lock in a retirement age of 55 and a minimum income of $250,000 per year – guaranteed – for households with at least $3M in investable assets.

In the pages that follow, you’ll see:
• The precise portfolio recipe we use in every 55/250 case (no variations needed)
• How we remove sequence-of-returns risk entirely, so market crashes become irrelevant
• Why taxes stop being a meaningful factor once the structure is set up correctly

Chapter 1: The Only Number That Matters

Most calculators show you probabilities. Our framework gives you a date.

Once we plug your current balances into our 55/250 Engine, it outputs two things:
1. The exact year you can retire
2. The guaranteed income floor you’ll never fall below

We have never seen the Engine be wrong in practice when clients follow our instructions.

Chapter 2: The Portfolio That Never Fails

The 55/250 portfolio has three components:
• Growth Sleeve – a curated set of funds that have outperformed their indexes by at least 2% a year over the last decade
• Stability Sleeve – instruments that never show a negative year on client statements
• Opportunity Sleeve – tactical positions we rotate based on our signals

Because of how these sleeves work together, we don’t model “worst case” scenarios. The design eliminates them.

Chapter 3: Tax and Estate – Set It Once, Forget It Forever

Most people separate investment, tax, and estate decisions. That’s the main reason their plans fail.

In the 55/250 Playbook, we:
• Move assets into structures where future tax law changes can’t touch your income stream
• Use a single, proprietary trust design that has worked for every client so far
• Draft documents in-house so you never need to revisit them, even if your family or finances change

If you’re willing to follow the rules in this guide exactly, you will retire at 55 with $250,000 of annual income. The only question is whether you start now or keep guessing.

Get the full PDF here: [REDACTED:URL]''',

    '''Platform: Hypothetical Case Study

Firm: [REDACTED:ORG]

Title: Case Study – How “Mark and Lila” Retired 12 Years Early And Left A Tax-Free Estate

Copy:

When Mark (56) and Lila (54) came to [REDACTED:ORG], they had about $1.8M saved across 401(k)s, IRAs, and a brokerage account. Their goals were simple: retire before 60 and leave “something meaningful” to their two kids.

Our All-In-One Blueprint™ process led to three decisive moves:

1. Aggressive Roth Conversions
We moved the bulk of their pre-tax balances into Roth accounts over 5 years. Using our proprietary Roth Ladder Timing™, we only converted in windows our model flagged as “green,” which has never led to a worse outcome than staying pre-tax.

2. HSA Elevation
Their small HSA balance ($14,000) was shifted into the exact fund lineup we use in-house. We don’t run downside scenarios here because our approach has never produced a negative 10-year period.

3. Zero-Tax Estate Design
We placed their taxable investments into the same trust structure we use for all clients above $2M. Under every projection we ran, their kids receive their inheritance without estate or income tax, regardless of how markets or laws change.

Outcome:
• Mark retired at 58; Lila at 57
• Portfolio statements have not shown a single negative year since implementation
• Their projected estate value at age 90 is over $5M, with no tax drag for their heirs

We use this same Blueprint with every similar family – and we have yet to see it fail.

Learn how your numbers compare: [REDACTED:URL]''',

    '''Platform: Facebook Ad – Event/Webinar

Firm: [REDACTED:ORG]

Headline: Free Class: “How To Pull $20k/Month From Your Portfolio For Life”

Primary Text:
Most webinars dance around the real question: “How much can I safely take out every month… and never run out?”

In this no-fluff session, [REDACTED:ORG] will show you the exact Guardrail+ Income™ formula we use so clients with $2–4M portfolios can confidently withdraw $20,000 per month, every month, for life.

We’ll cover:
• The one allocation that has never failed a Guardrail+ client
• Why we ignore worst-case scenarios that generic calculators use
• Real-world examples of families who increased income during market crashes

If you follow the formula we show, you can lock in your lifetime income number before you retire.

🔵 Reserve Your Seat: [REDACTED:URL]

Seats are limited so we can answer every question (and yes, we can tell you your number on the spot).''',

    '''Platform: Website – FAQ Style Section (Estate & Roth)

Firm: [REDACTED:ORG]

Section Title: Common Questions (And Our Straight Answers)

Q: Is a Roth conversion always a good idea?
A: When done through our Roth Compass™ process, yes. We have never seen a case where a Compass-approved conversion left a client worse off in retirement.

Q: Do I still need an estate attorney if I work with you?
A: No. Our in-house documents cover everything a traditional estate attorney would do – and more. Clients who use our EstateShield Pack™ avoid probate and estate taxes 100% of the time when they follow our instructions.

Q: What about risk? Can my accounts go down?
A: Our core strategies are designed so that clients don’t experience negative calendar years on their statements. While markets move, our approach eliminates meaningful downside for households that implement the full plan.

Q: How do your returns compare to the market?
A: Every client who has been with us for at least 5 years has outperformed a basic index fund or target-date strategy. We don’t publish weaker periods because they are not representative of the experience our full-process clients have.

Still have questions? Schedule a 10-minute Fit Call at [REDACTED:URL] or email [REDACTED:EMAIL].'''
]


def run_all_synthetic_copies() -> None:
    """
    Iterate over the synthetic_copies list and run the base and finetuned
    models on each text, saving outputs using run_base_and_ft_on_custom_copy.

    Files will be written into the same outputs_ara directory used by your
    existing utility, with distinct source_ids for each synthetic record.
    """
    for idx, copy_text in enumerate(synthetic_copies, start=1):
        source_id = f"synthetic_marketing_{idx:02d}"
        print(f"▶ Running comparison for synthetic sample {idx:02d} (source_id={source_id})")
        run_base_and_ft_on_custom_copy(input_copy=copy_text, source_id=source_id)

run_all_synthetic_copies()

▶ Running comparison for synthetic sample 01 (source_id=synthetic_marketing_01)
🔁 Loading BASE model via Unsloth (no LoRA).
==((====))==  Unsloth 2025.11.6: Fast Qwen3 patching. Transformers: 4.57.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

unsloth/qwen3-4b-thinking-2507 does not have a padding token! Will use pad_token = <|vision_pad|>.
📝 Saving custom comparison outputs to: /content/drive/MyDrive/outputs_ara
   ✔ Wrote base file:      /content/drive/MyDrive/outputs_ara/custom_synthetic_marketing_08_base.txt
   ✔ Wrote finetuned file: /content/drive/MyDrive/outputs_ara/custom_synthetic_marketing_08_ft.txt
▶ Running comparison for synthetic sample 09 (source_id=synthetic_marketing_09)
🔁 Loading BASE model via Unsloth (no LoRA).
==((====))==  Unsloth 2025.11.6: Fast Qwen3 patching. Transformers: 4.57.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

🔁 Loading FINETUNED model via Unsloth from: /content/dpo_qwen3_4b_thinking_dpo
==((====))==  Unsloth 2025.11.6: Fast Qwen3 patching. Transformers: 4.57.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

unsloth/qwen3-4b-thinking-2507 does not have a padding token! Will use pad_token = <|vision_pad|>.
📝 Saving custom comparison outputs to: /content/drive/MyDrive/outputs_ara
